In [ ]:
# import getpass
# import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

### Modelos

LangChain permite o uso de diferentes modelos de linguagem, tanto de código aberto quanto comerciais (como OpenAI, Cohere, Anthropic, etc.).

Os mais comuns:

- `ChatOpenAI`: para modelos de chat da OpenAI (`gpt-3.5-turbo`, `gpt-4`, etc.)
- `OpenAI`: para modelos não conversacionais (completions)
- `HuggingFaceHub`: para modelos hospedados no Hugging Face
- `ChatAnthropic`, `ChatCohere`, etc.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
response = llm.invoke("Olá")

print(response)

## Prompts

O prompt é o texto de entrada enviado ao modelo.

### Templates Simples

Usamos `PromptTemplate` para criar templates reutilizáveis com variáveis.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Traduza o seguinte texto para português: {texto}")

prompt = prompt_template.invoke({"texto": "Artificial Intelligence is the future!"})
print(prompt)

In [ ]:
response = llm.invoke(prompt)

print(response.content)

### Templates de Chat

Para modelos de chat, usamos `ChatPromptTemplate`.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
    HumanMessage(content="Hello, how are you?"),
]

# messages = [
#     ("system", "Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
#     ("human", "Hello, how are you?"),
# ]

response = llm.invoke(messages)

print(response)

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor de {lingua_origem} para {lingua_destino}. Traduza as mensagens que forem enviadas."),
        ("user", "{texto}")
    ]
)

In [ ]:
prompt = prompt_template.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "português",
    "texto": "Hello, how are you?"
})

print(prompt)

In [ ]:
response = llm.invoke(prompt)

print(response.content)

## Output Parsers

Os **parsers de saída** transformam a resposta do LLM em formatos úteis: JSON, dicionários, Pydantic, etc.

Eles ajudam a garantir que a resposta seja estruturada corretamente, facilitando a integração com código.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()

response = llm.invoke("Qual a capital do Rio Grande do Norte?")
output = str_parser.invoke(response)

print("Resposta:")
print(response)
print()
print("Saída do parser:")
print(output)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

response = llm.invoke("Quais as massas e cargas das partículas que constituem o átomo? Responda no formato JSON em que cada chave seja o nome da partícula")
output = json_parser.invoke(response)

print("Resposta:")
print(response)
print()
print("Saída do parser:")
print(output)

## 🔗 Encadeamento

LangChain permite encadear componentes usando o operador `|`, formando um fluxo simples e modular.

O padrão mais comum é:

```python
PromptTemplate | LLM | OutputParser
````

Ou seja:

* O **prompt** gera o texto com base nas variáveis
* O **modelo** (LLM) responde ao prompt
* O **parser** transforma a resposta em formato estruturado (como dicionário ou objeto)

In [ ]:
chain = prompt_template | llm | StrOutputParser()

In [ ]:
response = chain.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "espanhol",
    "texto": "As praias de Recife tem tubarões!"
})

print(response)

In [ ]:
def translate(texto, lingua_origem, lingua_destino):
    response = chain.invoke({
        "lingua_origem": lingua_origem,
        "lingua_destino": lingua_destino,
        "texto": texto
    })
    return response

In [ ]:
output = translate("The beaches of Recife have sharks!", "inglês", "espanhol")

print(output)

## Saída Estruturada

Quando usamos modelos de linguagem, muitas vezes queremos que eles retornem respostas em um **formato específico**, como um dicionário ou JSON com campos pré-definidos.

In [ ]:
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str = Field(description="O nome da pessoa presente no texto.")
    age: int = Field(description="A idade da pessoa presente no texto.")

In [ ]:
structured_llm = llm.with_structured_output(Person)

In [ ]:
SYSTEM_PROMPT = """Você é um assistente analisa textos e extrai informações sobre pessoas. Um texto será enviado e você deve extrair o nome e a idade da pessoa mencionada.

Texto: {text}"""

prompt_template = ChatPromptTemplate.from_template(SYSTEM_PROMPT)

chain = prompt_template | structured_llm

In [ ]:
response = chain.invoke({"text": "Certo dia João foi ao mercado e comprou um bolo para comemorar seu trigésimo aniversário."})

print(response)

## Exercícios

### Exercício 1
Crie uma `chain` que a partir de um tópico informado pelo usuário, crie uma piada.

### Exercício 2
Crie uma `chain` que classifique o sentimento de um texto de entrada em positivo, neutro ou negativo.

### Exercício 3
Crie uma `chain` que gere o código de uma função Python de acordo com a descrição do usuário.

### Exercício 4
Crie uma `chain` que explique de forma simplificada um tópico geral fornecido pelo usuário e, em seguida, traduza a explicação para inglês. Utilize dois templates encadeados.

### Exercício 5 - Desafio
Crie uma `chain` que responda perguntas sobre o CESAR School.